In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, TensorDataset
import random
import numpy as np
from sklearn.decomposition import PCA
from typing import Tuple
from tqdm import tqdm
from scipy.spatial import distance_matrix
import pandas as pd

In [7]:
root = "/Users/kristiansjorslevnielsen/Documents/DVP7/Features/"
# load data
X_hpo = torch.tensor(np.load(root + "X_hpo_Img.npy" ))                       
y_hpo = np.load(root + "y_hpo_Img.npy" )

X_valid = torch.tensor( np.load(root + "X_val_Img.npy" ) )
y_valid = np.load(root + "y_val_Img.npy")

print(X_hpo.shape)
print(y_hpo.shape)
print(X_valid.shape)
print(y_valid.shape)

X_hpo_tensor = torch.tensor(X_hpo, dtype=torch.long)
y_hpo_tensor = torch.tensor(y_hpo, dtype=torch.long)

X_valid_tensor = torch.tensor(X_valid, dtype=torch.long)
y_valid_tensor = torch.tensor(y_valid, dtype=torch.long)

hpo_dataset = TensorDataset(X_hpo_tensor, y_hpo_tensor)
valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor)

hpo_loader = DataLoader(hpo_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True)


torch.Size([50000, 4096])
(50000,)
torch.Size([5000, 4096])
(5000,)


/var/folders/9k/3q09wnl130996d8g7j3bwgbr0000gn/T/ipykernel_43331/1282788362.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_hpo_tensor = torch.tensor(X_hpo, dtype=torch.long)
/var/folders/9k/3q09wnl130996d8g7j3bwgbr0000gn/T/ipykernel_43331/1282788362.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_valid_tensor = torch.tensor(X_valid, dtype=torch.long)


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SUBIC_encoder(nn.Module): 
    def __init__(self, input_size=4096, bits=48, num_classes=10, num_blocks=8, block_size=6):
        super(SUBIC_encoder, self).__init__()
       
        assert bits % num_blocks == 0, "Bits must be divisible by num_blocks"

        self.input_size = input_size
        self.bits = bits 
        self.num_blocks = num_blocks
        self.block_size = block_size
        
        # Define the encoder structure
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 256), 
            nn.ReLU(),
            nn.Linear(256, bits)
        )  # Outputs binary feature vectors
        
        self.fc3 = nn.Linear(bits, num_classes)  # Logits for num_classes
    
    def block_softmax(self, x):
        
        batch_size = x.shape[0]
        block_size = x.shape[1] // self.num_blocks
        
        # Ensure that x has the expected shape
        assert x.shape[1] == self.bits, f"Expected shape [batch_size, {self.bits}], got {x.shape}"
        
        # Reshape and apply softmax
        x = x.view(batch_size, self.num_blocks, block_size)
        x = F.softmax(x, dim=-1) 
        return x.view(batch_size, -1) #-1 refers to the value that will match the original elements 
    
    def block_one_hot(self, x):
        batch_size = x.shape[0]

        x = x.view(batch_size, self.num_blocks, self.block_size)
        max_indices = x.argmax(dim=-1, keepdim=True)
        
        # Create one-hot encoding
        one_hot = torch.zeros_like(x).scatter_(-1, max_indices, 1)

        return one_hot.view(batch_size, self.bits)
    
    def forward(self, x, use_one_hot=False):
        # Ensure x is a flat tensor before passing to encoder
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)  # Flatten if necessary

        z = self.encoder(x)

        if use_one_hot:
            binary_codes = self.block_one_hot(z)
        else:
            binary_codes = self.block_softmax(z)

        class_probs = F.softmax(self.fc3(binary_codes), dim=-1) 

        return class_probs, binary_codes


In [28]:
from itertools import product # create cartesian products for all params

bits_12_param_grid = {
'bits':12,
'num_blocks':
[3, 4],
'gamma':[0.2, 0.6, 0.8],
'mu':[0.2, 0.6, 0.8],
'learning_rate': [0.005, 0.01]
}

bits_24_param_grid = {
'bits': 24,
'num_blocks':
[3, 6, 12],
'gamma':[0.2, 0.6, 0.8],
'mu':[0.2, 0.6, 0.8],
'learning_rate': [0.005, 0.01]
}

bits_32_param_grid = {
'bits':32,
'num_blocks':
[4, 8, 16],
'gamma':[0.2, 0.6, 0.8],
'mu':[0.2, 0.6, 0.8],
'learning_rate': [0.005, 0.01]
}

bits_48_param_grid = {
'bits':48,
'num_blocks':
[6, 12, 24],
'gamma':[0.2, 0.6, 0.8],
'mu':[0.2, 0.6, 0.8],
'learning_rate': [0.005, 0.01]
}

def generate_combinations(param_grid):
    keys = list(param_grid.keys())
    values = [v if isinstance(v, list) else [v] for v in param_grid.values()] # make list of dicts, if value is a list then take value from the list if not then take that scalar value
    gamma_idx = keys.index("gamma")
    mu_idx = keys.index("mu")
    gamma_mu_pairs = list(zip(values[gamma_idx], values[mu_idx]))
    combined_values = values[:gamma_idx] + [gamma_mu_pairs] + values[mu_idx+1:]
    combined_keys = keys[:gamma_idx] + ["gamma_mu"] + keys[mu_idx+1:]
    comb = list(product(*combined_values))
    param_combinations_dicts = [
        {
            **dict(zip(combined_keys, comb_item)),
            'gamma': comb_item[combined_keys.index('gamma_mu')][0],
            'mu': comb_item[combined_keys.index('gamma_mu')][1]
        } for comb_item in comb
    ]
    for comb_dict in param_combinations_dicts:
        del comb_dict['gamma_mu']
    
    return param_combinations_dicts

In [29]:
generate_combinations(bits_12_param_grid)

[{'bits': 12,
  'num_blocks': 3,
  'learning_rate': 0.005,
  'gamma': 0.2,
  'mu': 0.2},
 {'bits': 12, 'num_blocks': 3, 'learning_rate': 0.01, 'gamma': 0.2, 'mu': 0.2},
 {'bits': 12,
  'num_blocks': 3,
  'learning_rate': 0.005,
  'gamma': 0.6,
  'mu': 0.6},
 {'bits': 12, 'num_blocks': 3, 'learning_rate': 0.01, 'gamma': 0.6, 'mu': 0.6},
 {'bits': 12,
  'num_blocks': 3,
  'learning_rate': 0.005,
  'gamma': 0.8,
  'mu': 0.8},
 {'bits': 12, 'num_blocks': 3, 'learning_rate': 0.01, 'gamma': 0.8, 'mu': 0.8},
 {'bits': 12,
  'num_blocks': 4,
  'learning_rate': 0.005,
  'gamma': 0.2,
  'mu': 0.2},
 {'bits': 12, 'num_blocks': 4, 'learning_rate': 0.01, 'gamma': 0.2, 'mu': 0.2},
 {'bits': 12,
  'num_blocks': 4,
  'learning_rate': 0.005,
  'gamma': 0.6,
  'mu': 0.6},
 {'bits': 12, 'num_blocks': 4, 'learning_rate': 0.01, 'gamma': 0.6, 'mu': 0.6},
 {'bits': 12,
  'num_blocks': 4,
  'learning_rate': 0.005,
  'gamma': 0.8,
  'mu': 0.8},
 {'bits': 12, 'num_blocks': 4, 'learning_rate': 0.01, 'gamma': 0.8

In [30]:
def one_hot_encode(a):
    
    if isinstance(a, torch.Tensor):
         a = a.cpu().numpy()
    b = np.zeros((a.size, a.max() + 1))
    b[np.arange(a.size), a] = 1
    
    return b
def meanAveragePrecision(test_hashes, training_hashes, test_labels, training_labels):
    aps = []
    if len(training_labels.shape) == 1:
        training_labels = one_hot_encode(training_labels)
        test_labels = one_hot_encode(test_labels)
    for i, test_hash in enumerate(tqdm(test_hashes)):
        label = test_labels[i]
        distances = np.abs(training_hashes - test_hashes[i]).sum(axis=1)
        tp = np.where((training_labels*label).sum(axis=1)>0, 1, 0)
        hash_df = pd.DataFrame({"distances":distances, "tp":tp}).reset_index()
        hash_df = hash_df.sort_values(["distances", "index"]).reset_index(drop=True)
        hash_df = hash_df.drop(["index", "distances"], axis=1).reset_index()
        hash_df = hash_df[hash_df["tp"]==1]
        hash_df["tp"] = hash_df["tp"].cumsum()
        hash_df["index"] = hash_df["index"] +1 
        precision = np.array(hash_df["tp"]) / np.array(hash_df["index"])
        ap = precision.mean()
        aps.append(ap)
    
    return np.array(aps).mean()

In [31]:
def loss_function(epochs, bits, num_blocks, block_size, gamma, mu, learning_rate):
    # Initialize model
    model = SUBIC_encoder(
        bits=bits,
        input_size=X_hpo.shape[1],
        num_classes=10,
        num_blocks=num_blocks,
        block_size=block_size
    )
    model.to(device)  # Move model to the specified device
    
    # Initialize optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Track losses
    losses = []

    for epoch in range(epochs):
        model.train()  # Set model to training mode
        total_loss = 0.0

        for images, labels in hpo_loader:
            images, labels = images.to(device), labels.to(device)
            images, labels = images.to(torch.float), labels.to(torch.long)

            # Forward pass
            class_probs, binary_codes = model(images, use_one_hot=False)

            # Compute loss
            loss = compute_total_loss(
                class_probs, labels, binary_codes, num_blocks, block_size, gamma, mu
            )

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # Accumulate loss
            total_loss += loss.item()

        # Compute average loss for the epoch
        avg_loss = total_loss / len(hpo_loader)
        losses.append(avg_loss)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")

    return model, losses

def map_on_call(model):
    
    model.eval()
    all_query_codes, all_query_labels = [], []
    all_db_codes, all_db_labels = [], []

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            images, labels = images.to(torch.float), labels.to(torch.long)

            _, binary_codes = model(images, use_one_hot=True)

            # Ensure binary_codes is a tensor
            if not isinstance(binary_codes, torch.Tensor):
                raise TypeError("Expected binary_codes to be a tensor.")
            
            all_db_codes.append(binary_codes)
            all_db_labels.append(labels)
            if len(all_query_codes) == 0:  
                all_query_codes.append(binary_codes.clone())  
                all_query_labels.append(labels.clone())

    # Concatenate all tensors
    all_query_codes = torch.cat(all_query_codes, dim=0)
    all_query_labels = torch.cat(all_query_labels, dim=0)
    all_db_codes = torch.cat(all_db_codes, dim=0)
    all_db_labels = torch.cat(all_db_labels, dim=0)

    # Calculate MAP Score
    map_score = meanAveragePrecision(
        all_query_codes,
        all_db_codes,
        all_query_labels,
        all_db_labels
        )

    return map_score 
        

In [32]:
bits_12_param_dicts = generate_combinations(bits_12_param_grid) 
bits_24_param_dicts = generate_combinations(bits_24_param_grid) 
bits_32_param_dicts = generate_combinations(bits_32_param_grid) 
bits_48_param_dicts = generate_combinations(bits_48_param_grid) 

In [33]:
results = []
epochs = 100
map_results = {}

# iterate over each combination, train the model and evaluate its performance 
# initailize model with current hyperparameters

def hyper_tuning(param_grid, epochs, hpo_train, hpo_loader, device):
    results = {}  # To store MAP scores for each hyperparameter set
    best_map_score = 0  # Initialize with a very low value
    best_params = None  # To store the best parameter set

    for params in param_grid:
        # Extract parameters from the current set
        bits = params['bits']
        num_blocks = params['num_blocks']
        block_size = bits // num_blocks
        gamma = params['gamma']
        mu = params['mu']
        learning_rate = params['learning_rate']

        print(f"Testing hyperparameters: {params}")

        # Initialize model with the current hyperparameters
        model = SUBIC_encoder(bits=bits, input_size= hpo_train.shape[1], num_classes=10, num_blocks=num_blocks, block_size=block_size)
        model.to(device) 

        # Train the model with the current parameter set
        trained_model, _ = loss_function(epochs, bits, num_blocks, block_size, gamma, mu, learning_rate)

        # Evaluate the model using MAP score
        map_score = map_on_call(trained_model)
        print(f"MAP score: {map_score:.4f}")

        # Store the MAP score for the current parameter set
        results[tuple(params.items())] = map_score

        if map_score > best_map_score:
            best_map_score = map_score
            best_params = params

    print(f"Best Hyperparameters: {best_params}, Best MAP Score: {best_map_score:.4f}")
    return best_params, best_map_score, results




In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
losses = []
maps = []

hyper_tuning(param_grid=bits_12_param_dicts, epochs=100, hpo_train=X_hpo, hpo_loader=hpo_loader, device=device)

Testing hyperparameters: {'bits': 12, 'num_blocks': 3, 'learning_rate': 0.005, 'gamma': 0.2, 'mu': 0.2}


NameError: name 'compute_total_loss' is not defined

In [ ]:
hyper_tuning(param_grid=bits_24_param_dicts, epochs=100, hpo_train=X_hpo, hpo_loader=hpo_loader, device=device)

Testing hyperparameters: {'bits': 24, 'num_blocks': 3, 'gamma': 0.2, 'mu': 0.2, 'learning_rate': 0.005}
Epoch 1/100, Loss: 0.7965
Epoch 2/100, Loss: 0.6030
Epoch 3/100, Loss: 0.5161
Epoch 4/100, Loss: 0.4979
Epoch 5/100, Loss: 0.5102
Epoch 6/100, Loss: 0.4793
Epoch 7/100, Loss: 0.4449
Epoch 8/100, Loss: 0.4805
Epoch 9/100, Loss: 0.4733
Epoch 10/100, Loss: 0.4278
Epoch 11/100, Loss: 0.4476
Epoch 12/100, Loss: 0.4259
Epoch 13/100, Loss: 0.4068
Epoch 14/100, Loss: 0.4147
Epoch 15/100, Loss: 0.4705
Epoch 16/100, Loss: 0.4227
Epoch 17/100, Loss: 0.4418
Epoch 18/100, Loss: 0.4077
Epoch 19/100, Loss: 0.4573
Epoch 20/100, Loss: 0.4167
Epoch 21/100, Loss: 0.4585
Epoch 22/100, Loss: 0.4177
Epoch 23/100, Loss: 0.4147
Epoch 24/100, Loss: 0.4147
Epoch 25/100, Loss: 0.4000
Epoch 26/100, Loss: 0.3765
Epoch 27/100, Loss: 0.3647
Epoch 28/100, Loss: 0.3683
Epoch 29/100, Loss: 0.3691
Epoch 30/100, Loss: 0.4155
Epoch 31/100, Loss: 0.3826
Epoch 32/100, Loss: 0.3785
Epoch 33/100, Loss: 0.3832
Epoch 34/100, 

100%|██████████| 64/64 [00:00<00:00, 95.84it/s] 


MAP score: 0.3652
Testing hyperparameters: {'bits': 24, 'num_blocks': 3, 'gamma': 0.2, 'mu': 0.2, 'learning_rate': 0.01}
Epoch 1/100, Loss: 1.0020
Epoch 2/100, Loss: 1.0007
Epoch 3/100, Loss: 1.0016
Epoch 4/100, Loss: 1.0011
Epoch 5/100, Loss: 1.0017
Epoch 6/100, Loss: 1.0014
Epoch 7/100, Loss: 1.0018
Epoch 8/100, Loss: 1.0015
Epoch 9/100, Loss: 1.0015
Epoch 10/100, Loss: 1.0013
Epoch 11/100, Loss: 1.0017
Epoch 12/100, Loss: 1.0016
Epoch 13/100, Loss: 1.0018
Epoch 14/100, Loss: 1.0019
Epoch 15/100, Loss: 1.0009
Epoch 16/100, Loss: 1.0016
Epoch 17/100, Loss: 1.0015
Epoch 18/100, Loss: 1.0010
Epoch 19/100, Loss: 1.0015
Epoch 20/100, Loss: 1.0013
Epoch 21/100, Loss: 1.0021
Epoch 22/100, Loss: 1.0013
Epoch 23/100, Loss: 1.0018
Epoch 24/100, Loss: 1.0016
Epoch 25/100, Loss: 1.0023
Epoch 26/100, Loss: 1.0018
Epoch 27/100, Loss: 1.0018
Epoch 28/100, Loss: 1.0019
Epoch 29/100, Loss: 1.0013
Epoch 30/100, Loss: 1.0014
Epoch 31/100, Loss: 1.0017
Epoch 32/100, Loss: 1.0015
Epoch 33/100, Loss: 1.00

100%|██████████| 64/64 [00:00<00:00, 117.99it/s]


MAP score: 0.1147
Testing hyperparameters: {'bits': 24, 'num_blocks': 3, 'gamma': 0.2, 'mu': 0.2, 'learning_rate': 0.05}
Epoch 1/100, Loss: 1.0088
Epoch 2/100, Loss: 1.0082
Epoch 3/100, Loss: 1.0093
Epoch 4/100, Loss: 1.0074
Epoch 5/100, Loss: 1.0074
Epoch 6/100, Loss: 1.0085
Epoch 7/100, Loss: 1.0073
Epoch 8/100, Loss: 1.0083
Epoch 9/100, Loss: 1.0076
Epoch 10/100, Loss: 1.0074
Epoch 11/100, Loss: 1.0061
Epoch 12/100, Loss: 1.0075
Epoch 13/100, Loss: 1.0093
Epoch 14/100, Loss: 1.0068
Epoch 15/100, Loss: 1.0090
Epoch 16/100, Loss: 1.0084
Epoch 17/100, Loss: 1.0083
Epoch 18/100, Loss: 1.0091
Epoch 19/100, Loss: 1.0099
Epoch 20/100, Loss: 1.0076
Epoch 21/100, Loss: 1.0095
Epoch 22/100, Loss: 1.0071
Epoch 23/100, Loss: 1.0079
Epoch 24/100, Loss: 1.0061
Epoch 25/100, Loss: 1.0075
Epoch 26/100, Loss: 1.0063
Epoch 27/100, Loss: 1.0087
Epoch 28/100, Loss: 1.0067
Epoch 29/100, Loss: 1.0073
Epoch 30/100, Loss: 1.0096
Epoch 31/100, Loss: 1.0114
Epoch 32/100, Loss: 1.0062
Epoch 33/100, Loss: 1.00

100%|██████████| 64/64 [00:00<00:00, 240.05it/s]


MAP score: 0.1135
Testing hyperparameters: {'bits': 24, 'num_blocks': 3, 'gamma': 0.2, 'mu': 0.6, 'learning_rate': 0.005}
Epoch 1/100, Loss: 0.6626
Epoch 2/100, Loss: 0.4429
Epoch 3/100, Loss: 0.3772
Epoch 4/100, Loss: 0.3171
Epoch 5/100, Loss: 0.3208
Epoch 6/100, Loss: 0.2107
Epoch 7/100, Loss: 0.1884
Epoch 8/100, Loss: 0.1792
Epoch 9/100, Loss: 0.1735
Epoch 10/100, Loss: 0.1517
Epoch 11/100, Loss: 0.1600
Epoch 12/100, Loss: 0.1840
Epoch 13/100, Loss: 0.1820
Epoch 14/100, Loss: 0.1202
Epoch 15/100, Loss: 0.1173
Epoch 16/100, Loss: 0.1371
Epoch 17/100, Loss: 0.1423
Epoch 18/100, Loss: 0.1177
Epoch 19/100, Loss: 0.1174
Epoch 20/100, Loss: 0.1219
Epoch 21/100, Loss: 0.1223
Epoch 22/100, Loss: 0.1108
Epoch 23/100, Loss: 0.0992
Epoch 24/100, Loss: 0.1414
Epoch 25/100, Loss: 0.0896
Epoch 26/100, Loss: 0.0836
Epoch 27/100, Loss: 0.0587
Epoch 28/100, Loss: 0.0721
Epoch 29/100, Loss: 0.1264
Epoch 30/100, Loss: 0.0948
Epoch 31/100, Loss: 0.0995
Epoch 32/100, Loss: 0.0458
Epoch 33/100, Loss: 0.0

KeyboardInterrupt: 

In [ ]:
param_grid=bits_24_param_dicts

In [ ]:
test_param = [{'bits': 12,
  'num_blocks': 3,
  'gamma': 0.6,
  'mu': 0.6,
  'learning_rate': 0.005}]

In [ ]:
hyper_tuning(param_grid=test_param, epochs=100, hpo_train=X_hpo, hpo_loader=hpo_loader, device=device)

Testing hyperparameters: {'bits': 12, 'num_blocks': 3, 'gamma': 0.6, 'mu': 0.6, 'learning_rate': 0.005}
Epoch 1/100, Loss: 0.6518
Epoch 2/100, Loss: 0.5493
Epoch 3/100, Loss: 0.4896
Epoch 4/100, Loss: 0.3813
Epoch 5/100, Loss: 0.3448
Epoch 6/100, Loss: 0.4755
Epoch 7/100, Loss: 0.4314
Epoch 8/100, Loss: 0.4422
Epoch 9/100, Loss: 0.4568
Epoch 10/100, Loss: 0.4816
Epoch 11/100, Loss: 0.4509
Epoch 12/100, Loss: 0.4206
Epoch 13/100, Loss: 0.4327
Epoch 14/100, Loss: 0.3642
Epoch 15/100, Loss: 0.3379
Epoch 16/100, Loss: 0.3517
Epoch 17/100, Loss: 0.2954
Epoch 18/100, Loss: 0.3123
Epoch 19/100, Loss: 0.3140
Epoch 20/100, Loss: 0.3223
Epoch 21/100, Loss: 0.2715
Epoch 22/100, Loss: 0.2598
Epoch 23/100, Loss: 0.2782
Epoch 24/100, Loss: 0.3077
Epoch 25/100, Loss: 0.2872
Epoch 26/100, Loss: 0.2648
Epoch 27/100, Loss: 0.3181
Epoch 28/100, Loss: 0.2628
Epoch 29/100, Loss: 0.3023
Epoch 30/100, Loss: 0.2684
Epoch 31/100, Loss: 0.2423
Epoch 32/100, Loss: 0.2907
Epoch 33/100, Loss: 0.3374
Epoch 34/100, 

100%|██████████| 64/64 [00:00<00:00, 166.89it/s]

MAP score: 0.3341
Best Hyperparameters: {'bits': 12, 'num_blocks': 3, 'gamma': 0.6, 'mu': 0.6, 'learning_rate': 0.005}, Best MAP Score: 0.3341


({'bits': 12,
  'num_blocks': 3,
  'gamma': 0.6,
  'mu': 0.6,
  'learning_rate': 0.005},
 0.33414062869259775,
 {(('bits', 12),
   ('num_blocks', 3),
   ('gamma', 0.6),
   ('mu', 0.6),
   ('learning_rate', 0.005)): 0.33414062869259775})